# Практика
Используемые библиотеки

In [33]:
import pandas as pd
import numpy as np
import requests # for web-download
import io # for web-download
import re # for data processing
import random

## Загрузка DataFrame
### Задача 1
На основании данных портала "Открытые данные России" о результатах Химического анализа родника в Нескучном саду https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad
средствами библиотеки __Pandas__ сформируйте поле выводов по каждому анализирумомому параметру.
Например, по показателю _pH_ получен результат _8.4 единицы pH_ при нормативе от _6 до 9 единиц pH_. Т.о. по данному показателю результат анализа в норме.
Для решения задачи необходимо программно "прочитать и понять" значение столбца "Норматив" и выделенное численное значение сравнить с нормативом согласно логике норматива. Например, __6 >= pH >= 9__.
В итоговом DataFrame столбец "Показатель" сделайте индексным.


Загзрузка DataFrame выполняется непосредственно c сайта "Открытые данные России" https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv (см. код ниже).


In [45]:
headers={
    'User-Agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:52.0) Gecko/20100101 Firefox/52.0',
}
def get_content(url):
    with requests.Session() as req:
        req.headers.update(headers)
        r = req.get(url).content
    return r

url ="https://data.gov.ru/opendata/7708660670-rodnik-neskuchniy-sad/data-20160608T1215-structure-20160608T1215.csv"
s = get_content(url)
df=pd.read_csv(io.StringIO(s.decode('UTF8')))

display(df)

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.4,в пределах 6-9
1,Запах,баллы,1,не более 2-3
2,Цветность,градусы,б/цвета,не более 30
3,Жёсткость,мг-эквл/дм3,9.199999999999999,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.42,"не более 1,5"
5,Нитриты (по NO2),мг/дм3,0.017,"не более 3,3"
6,Нитраты (по NO3),мг/дм3,24,не более 45
7,Фосфаты (P),мг/дм3,0.36,"не более 3,5"
8,Хлориды (Cl),мг/дм3,200,не более 350
9,Сульфаты (SO4),мг/дм3,189.5,не более 500


Проанализируем столбец результат анализа. Все значения в столбце преставлены в виде числового значения, кроме значения по цветности. Заменим данное значение на 0.

In [46]:
df['Результат анализа'] = df['Результат анализа'].replace(to_replace=r'б/цвета', value='0', regex=True)

In [47]:
#приведем стобец Результат анализа к числу с плавающей точкой
df['Результат анализа'] = df['Результат анализа'].astype(float)
#в столбце Норматив запятую заменим на точку, для удобства дальнейших вычислений
df["Норматив"] = df["Норматив"].str.replace(',', '.')
df

,Показатель,Единица измерений,Результат анализа,Норматив
0,pH,единицы pH,8.400,в пределах 6-9
1,Запах,баллы,1.000,не более 2-3
2,Цветность,градусы,0.000,не более 30
3,Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,не более 1.5
5,Нитриты (по NO2),мг/дм3,0.017,не более 3.3
6,Нитраты (по NO3),мг/дм3,24.000,не более 45
7,Фосфаты (P),мг/дм3,0.360,не более 3.5
8,Хлориды (Cl),мг/дм3,200.000,не более 350
9,Сульфаты (SO4),мг/дм3,189.500,не более 500


In [48]:
#разобьем по пробелу значения в столбце Норматив
sep_norm = df['Норматив'].str.split(expand=True)
#разобьем по - граничные значения
val_gran = sep_norm[2].str.split('-', expand=True)
df['Критерий норматива'] = sep_norm[0]+' '+sep_norm[1]
df['Левая граница'] = val_gran[0]
df['Правая граница'] = val_gran[1]
df

,Показатель,Единица измерений,Результат анализа,Норматив,Критерий норматива,Левая граница,Правая граница
0,pH,единицы pH,8.400,в пределах 6-9,в пределах,6,9
1,Запах,баллы,1.000,не более 2-3,не более,2,3
2,Цветность,градусы,0.000,не более 30,не более,30,None
3,Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,в пределах,7,10
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,не более 1.5,не более,1.5,None
5,Нитриты (по NO2),мг/дм3,0.017,не более 3.3,не более,3.3,None
6,Нитраты (по NO3),мг/дм3,24.000,не более 45,не более,45,None
7,Фосфаты (P),мг/дм3,0.360,не более 3.5,не более,3.5,None
8,Хлориды (Cl),мг/дм3,200.000,не более 350,не более,350,None
9,Сульфаты (SO4),мг/дм3,189.500,не более 500,не более,500,None


In [49]:
#выделим значения соответсвующие null в правой границе
null = df['Правая граница'].isnull()
# и заменим в столбце Правая граница значения null из левой границы
df.loc[null, 'Правая граница'] = df.loc[null, 'Левая граница']
# и приведем все к типу float
df['Левая граница'] = df['Левая граница'].astype(float)
df['Правая граница'] = df['Правая граница'].astype(float)
df

,Показатель,Единица измерений,Результат анализа,Норматив,Критерий норматива,Левая граница,Правая граница
0,pH,единицы pH,8.400,в пределах 6-9,в пределах,6.0,9.0
1,Запах,баллы,1.000,не более 2-3,не более,2.0,3.0
2,Цветность,градусы,0.000,не более 30,не более,30.0,30.0
3,Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,в пределах,7.0,10.0
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,не более 1.5,не более,1.5,1.5
5,Нитриты (по NO2),мг/дм3,0.017,не более 3.3,не более,3.3,3.3
6,Нитраты (по NO3),мг/дм3,24.000,не более 45,не более,45.0,45.0
7,Фосфаты (P),мг/дм3,0.360,не более 3.5,не более,3.5,3.5
8,Хлориды (Cl),мг/дм3,200.000,не более 350,не более,350.0,350.0
9,Сульфаты (SO4),мг/дм3,189.500,не более 500,не более,500.0,500.0


In [50]:
#формирование результата
good = (df['Критерий норматива'] == 'не более') & (df['Правая граница'] > df['Результат анализа'])
bad = (df['Критерий норматива'] == 'не более') & (df['Правая граница'] < df['Результат анализа'])
good_variation = ((df['Критерий норматива'] == 'в пределах') & (df['Правая граница'] >= df['Результат анализа']) & (df['Левая граница'] <= df['Результат анализа']))
bad_variation = ((df['Критерий норматива'] == 'в пределах') & (df['Правая граница'] <= df['Результат анализа']) & (df['Левая граница'] <= df['Результат анализа']))

df.loc[good, 'Результат'] = 'Соответствует нормативу'
df.loc[good_variation, 'Результат'] = 'Соответствует нормативу'
df.loc[bad, 'Результат'] = '!!Не соответствет нормативу!!'
df.loc[bad_variation, 'Результат'] = '!!Не соответствет нормативу!!'
#удаление вспомогательных столбцов
df.drop(['Критерий норматива','Левая граница','Правая граница'],axis=1,inplace= True)
df

,Показатель,Единица измерений,Результат анализа,Норматив,Результат
0,pH,единицы pH,8.400,в пределах 6-9,Соответствует нормативу
1,Запах,баллы,1.000,не более 2-3,Соответствует нормативу
2,Цветность,градусы,0.000,не более 30,Соответствует нормативу
3,Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,Соответствует нормативу
4,Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,не более 1.5,Соответствует нормативу
5,Нитриты (по NO2),мг/дм3,0.017,не более 3.3,Соответствует нормативу
6,Нитраты (по NO3),мг/дм3,24.000,не более 45,Соответствует нормативу
7,Фосфаты (P),мг/дм3,0.360,не более 3.5,Соответствует нормативу
8,Хлориды (Cl),мг/дм3,200.000,не более 350,Соответствует нормативу
9,Сульфаты (SO4),мг/дм3,189.500,не более 500,Соответствует нормативу


In [51]:
#сделаем столбец "Показатель" индексным
df.set_index('Показатель')

,Единица измерений,Результат анализа,Норматив,Результат
Показатель,,,,
pH,единицы pH,8.400,в пределах 6-9,Соответствует нормативу
Запах,баллы,1.000,не более 2-3,Соответствует нормативу
Цветность,градусы,0.000,не более 30,Соответствует нормативу
Жёсткость,мг-эквл/дм3,9.200,в пределах 7-10,Соответствует нормативу
Аммиак и аммоний-ион (по азоту),мг/дм3,0.420,не более 1.5,Соответствует нормативу
Нитриты (по NO2),мг/дм3,0.017,не более 3.3,Соответствует нормативу
Нитраты (по NO3),мг/дм3,24.000,не более 45,Соответствует нормативу
Фосфаты (P),мг/дм3,0.360,не более 3.5,Соответствует нормативу
Хлориды (Cl),мг/дм3,200.000,не более 350,Соответствует нормативу


## Теория вероятности. События

Требуется сгенерировать необходимые выборки и произвести по ним расчеты

### Задача 2
В ящике 5 апельсинов и 4 яблока. Наудачу выбираются 3 фрукта. Какова вероятность, что все три фрукта – апельсины?

В интернете полученный аналитически ответ 0.119. Подтверждается ли он эксперементально?


In [29]:
# заполняем ящик апельсинами и яблоками и достаем первый апельсин
Box = ["orange"]*5 + ['apple']*4
# ящик без одного апельсина
Box1 = ["orange"]*4 + ['apple']*4
# ящик без двух апельсина
Box2 = ["orange"]*3 + ['apple']*4
# задаем число экспериментов
n = 100000
#формируем DF
A = pd.DataFrame({"B1":list(map(lambda a: random.choice(Box), range(n))),
                  "B2":list(map(lambda a: random.choice(Box1), range(n))),
                  "B3":list(map(lambda a: random.choice(Box2), range(n))),
                 })

# считаем вероятность, что все три фрукта – апельсины
P_orange_three = len(A.query('B1=="orange" & B2=="orange" & B3=="orange"'))/A.shape[0]
P_orange_three

0.11922

В результате решения задачи экспериментальный ответ совпадает с аналитическим ответом из интернета. 


### Задача 3
Мастер, имея 10 деталей, из которых 3 – нестандартных, проверяет детали одну за другой, пока ему не попадется стандартная. Какова вероятность, что он проверит ровно две детали?


В интернете полученный аналитически ответ 7/30 или 0.23333. Подтверждается ли он эксперементально?

In [37]:
# заполняем ящик деталями и достаем первую деталь
Box = ["standart"]*7 + ['nonstandart']*3
#достаем вторую деталь
Box1 = ["standart"]*6 + ['nonstandart']*3
# задаем число экспериментов
n = 100000
#формируем DF
A = pd.DataFrame({"B1":list(map(lambda a: random.choice(Box), range(n))),
                  "B2":list(map(lambda a: random.choice(Box1), range(n)))
                 })
# считаем вероятность, что он проверит ровно две детали
P_det_two = len(A.query('B1=="standart" & B2=="nonstandart"'))/A.shape[0]
P_det_two

0.23426

В результате решения задачи экспериментальный ответ совпадает с аналитическим ответом из интернета.